In [31]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# ==============================================================
# 1. Load Data & Define Standardization Functions
# ==============================================================
df = pd.read_excel('C:/Users/user/OneDrive/gym_recommendation_system/venv/data/gym recommendation.xlsx')

def standardize_text(text):
    """Clean and normalize text entries"""
    text = re.sub(r'[^a-zA-Z0-9 ]', '', str(text))  # Remove special chars
    text = text.strip().lower()
    # Manual term replacements
    replacements = {
        'dumbbells': 'dumbbell',
        'treadmills': 'treadmill',
        'swimming.': 'swimming',
        'carrots': 'carrot',
        'male': 0, 'female': 1,  # Predefine gender encoding
        'yes': 1, 'no': 0        # Predefine binary flags
    }
    return replacements.get(text, text)

# =================================================================
# 2. Process Multi-Label Columns
# =================================================================
def process_diet(diet_str):
    """Process diet column with nested structure"""
    diet_str = re.sub(r'\w+:\s*', '', str(diet_str)) 
    items = re.findall(r'\((.*?)\)', diet_str)
    return [standardize_text(item) 
            for group in items 
            for item in re.split(r', | and ', group) 
            if item.strip()]

def process_generic(column_str):
    """Process Exercises/Equipment columns"""
    return [standardize_text(item) 
            for item in re.split(r', | and ', str(column_str)) 
            if item.strip()]

# Apply processing
df['Diet'] = df['Diet'].apply(process_diet)
df['Exercises'] = df['Exercises'].apply(process_generic)
df['Equipment'] = df['Equipment'].apply(process_generic)

# =================================================================
# 3. Encode Multi-Label Features
# =================================================================
class DedupeMLB(MultiLabelBinarizer):
    """Custom MLB with text standardization"""
    def fit_transform(self, y):
        y_clean = [[standardize_text(item) for item in lst] for lst in y]
        return super().fit_transform(y_clean)

def safe_encode(df, column):
    """Encoding with automatic deduplication"""
    mlb = DedupeMLB()
    encoded = pd.DataFrame(
        mlb.fit_transform(df[column]),
        columns=[f"{column}_{item.replace(' ', '_')}" for item in mlb.classes_],
        index=df.index
    )
    return encoded.T.groupby(level=0).max().T

# Encode all multi-label columns
diet_encoded = safe_encode(df, 'Diet')
exercises_encoded = safe_encode(df, 'Exercises')
equipment_encoded = safe_encode(df, 'Equipment')

# =================================================================
# 4. Encode Categorical Variables
# =================================================================
# Convert binary columns
binary_cols = ['Sex', 'Hypertension', 'Diabetes']
df[binary_cols] = df[binary_cols].applymap(standardize_text)

# One-hot encode multi-class columns
cat_cols = ['Level', 'Fitness Goal', 'Fitness Type']
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe_features = ohe.fit_transform(df[cat_cols])
ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(cat_cols))

# =================================================================
# 5. Create Final DataFrame
# =================================================================
final_df = pd.concat([
    df.drop(['Diet', 'Exercises', 'Equipment', 'ID', 'Recommendation'] + cat_cols, axis=1),
    ohe_df,
    diet_encoded.add_prefix('diet_'),
    exercises_encoded.add_prefix('exercise_'),
    equipment_encoded.add_prefix('equipment_')
], axis=1).astype(float)

# =================================================================
# 6. Model Training (Revised)
# =================================================================
# Verify columns
print("Final features:", final_df.columns.tolist()[:30], "...")

# Isolate key features
feature_cols = ['Age', 'BMI', 'Hypertension', 'Fitness Goal_Weight Loss']
target_col = 'diet_Diet_and_lowfat_dairy_products'

X = final_df[feature_cols]
y = final_df[target_col]

# Retrain model
model = RandomForestClassifier()
model.fit(X, y)

# Show interpretable features
importance = pd.Series(model.feature_importances_, index=feature_cols)
print("\nKey Drivers of Diet Choices:")
print(importance.sort_values(ascending=False))

# =================================================================
# 7. Output Sample
# =================================================================
print("\nSample preprocessed data:")
print(final_df.iloc[:2, :5].T)

C:\Users\user\AppData\Local\Temp\ipykernel_536\2478514432.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(standardize_text)


Final features: ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Level_Obuse', 'Level_Overweight', 'Level_Underweight', 'Fitness Goal_Weight Loss', 'Fitness Type_Muscular Fitness', 'diet_Diet_aloe_vera_juice', 'diet_Diet_and_apple', 'diet_Diet_and_baru_nuts', 'diet_Diet_and_beetroot_juice', 'diet_Diet_and_bell_peppers', 'diet_Diet_and_black_walnut', 'diet_Diet_and_hemp_seeds', 'diet_Diet_and_iceberg_lettuce', 'diet_Diet_and_jelly_sandwich', 'diet_Diet_and_lettuce', 'diet_Diet_and_lowfat_dairy_products', 'diet_Diet_and_mango_juice', 'diet_Diet_and_mixed_teff', 'diet_Diet_and_nuts', 'diet_Diet_and_watermelon_juice', 'diet_Diet_apple_juice', 'diet_Diet_baru_nuts', 'diet_Diet_beech_nuts'] ...

Key Drivers of Diet Choices:
BMI                         0.520720
Hypertension                0.318700
Age                         0.121821
Fitness Goal_Weight Loss    0.038759
dtype: float64

Sample preprocessed data:
                  0      1
Sex            0.00   0.00
Age   

In [59]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pickle

# **Step 1: Define user and plan feature columns**
# Replace 'final_df' with your actual DataFrame
user_cols = ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI'] + \
            [col for col in final_df.columns if col.startswith(('Level_', 'Fitness Goal_'))]
plan_cols = [col for col in final_df.columns if col.startswith(('exercise_', 'equipment_', 'diet_', 'Fitness Type_'))]

# Ensure all columns are numeric
final_df = final_df.astype(float)

# **Step 2: Incorporate Key Drivers with Feature Weighting**
importance = {
    'BMI': 0.5399,              # High importance
    'Hypertension': 0.2947,     # Moderate importance
    'Age': 0.1200,              # Lower importance
    'Fitness Goal_Weight Loss': 0.0454  # Lowest importance
}

# Scale the key driver features by their importance scores
for col, weight in importance.items():
    if col in user_cols:
        final_df[col] *= weight

# **Step 3: Prepare Positive and Negative Examples**
user_features_pos = final_df[user_cols].values
plan_features_pos = final_df[plan_cols].values
labels_pos = np.ones(len(final_df))  # Positive pairs

# Generate negative examples
num_negatives = 2
neg_user_features = []
neg_plan_features = []
for i in range(len(final_df)):
    for _ in range(num_negatives):
        neg_idx = np.random.randint(0, len(final_df))
        while neg_idx == i:  # Ensure it's not the positive pair
            neg_idx = np.random.randint(0, len(final_df))
        neg_user_features.append(user_features_pos[i])
        neg_plan_features.append(plan_features_pos[neg_idx])

neg_user_features = np.array(neg_user_features)
neg_plan_features = np.array(neg_plan_features)
labels_neg = np.zeros(len(neg_user_features))  # Negative pairs

# Combine positive and negative examples
all_user_features = np.vstack([user_features_pos, neg_user_features])
all_plan_features = np.vstack([plan_features_pos, neg_plan_features])
all_labels = np.hstack([labels_pos, labels_neg])

# **Step 4: Split into Training and Validation Sets**
train_user, val_user, train_plan, val_plan, train_labels, val_labels = train_test_split(
    all_user_features, all_plan_features, all_labels, test_size=0.2, random_state=42
)

# Convert to PyTorch tensors
train_user_tensor = torch.tensor(train_user, dtype=torch.float32)
train_plan_tensor = torch.tensor(train_plan, dtype=torch.float32)
train_label_tensor = torch.tensor(train_labels, dtype=torch.float32)
val_user_tensor = torch.tensor(val_user, dtype=torch.float32)
val_plan_tensor = torch.tensor(val_plan, dtype=torch.float32)
val_label_tensor = torch.tensor(val_labels, dtype=torch.float32)

# **Step 5: Define the Two-Tower Model**
class TwoTowerModel(nn.Module):
    def __init__(self, user_feature_dim, plan_feature_dim, embedding_dim=64):
        super().__init__()
        self.user_encoder = nn.Sequential(
            nn.Linear(user_feature_dim, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )
        self.plan_encoder = nn.Sequential(
            nn.Linear(plan_feature_dim, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def forward(self, user_features, plan_features):
        user_emb = self.user_encoder(user_features)
        plan_emb = self.plan_encoder(plan_features)
        score = (user_emb * plan_emb).sum(dim=1)
        return torch.sigmoid(score)

# Initialize the model
model = TwoTowerModel(user_feature_dim=len(user_cols), plan_feature_dim=len(plan_cols))

# **Step 6: Train the Model and Track Performance Metrics**
train_dataset = TensorDataset(train_user_tensor, train_plan_tensor, train_label_tensor)
val_dataset = TensorDataset(val_user_tensor, val_plan_tensor, val_label_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

num_epochs = 10
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    for user_batch, plan_batch, label_batch in train_loader:
        optimizer.zero_grad()
        scores = model(user_batch, plan_batch)
        loss = criterion(scores, label_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation phase
    model.eval()
    val_scores = []
    val_true_labels = []
    with torch.no_grad():
        for user_batch, plan_batch, label_batch in val_loader:
            scores = model(user_batch, plan_batch)
            val_scores.extend(scores.numpy())
            val_true_labels.extend(label_batch.numpy())

    val_scores = np.array(val_scores)
    val_true_labels = np.array(val_true_labels)
    val_predictions = (val_scores > 0.5).astype(int)
    precision = precision_score(val_true_labels, val_predictions)
    recall = recall_score(val_true_labels, val_predictions)
    f1 = f1_score(val_true_labels, val_predictions)
    auc = roc_auc_score(val_true_labels, val_scores)

    print(f"Epoch {epoch+1}, Train Loss: {total_loss / len(train_loader):.4f}, "
          f"Val Precision: {precision:.4f}, Val Recall: {recall:.4f}, Val F1: {f1:.4f}, Val AUC: {auc:.4f}")

# **Step 7: Save the Model and Additional Data**
torch.save(model.state_dict(), 'recommendation_model.pth')

with open('model_data.pkl', 'wb') as f:
    pickle.dump({
        'user_cols': user_cols,
        'plan_cols': plan_cols,
        'importance': importance
    }, f)

print("Model and data saved successfully!")

# **Step 8: Define Nutritional Category Mapping**
# Customize this mapping based on your dataset
food_category_mapping = {
    'apple juice': 'Carbohydrates',
    'baru nuts': 'Protein',
    'beetroot juice': 'Vitamins',
    'squash seeds': 'Fiber',
    'broccoli': 'Fiber',
    'carrot': 'Fiber',
    'chicken': 'Protein',
    'fish': 'Protein',
    'garlic': 'Vitamins',
    'iceberg lettuce': 'Fiber',
    'mango juice': 'Carbohydrates',
    'Roma tomatoes': 'Vitamins',
    'spinach': 'Vitamins',
    'tofu': 'Protein',
    'lettuce': 'Fiber',
    'bell peppers': 'Vitamins',
    'mango': 'Carbohydrates',
    'cherry tomatoes': 'Vitamins',
    'poultry': 'Protein',
    'celery': 'Fiber',
    'coldpressed juice': 'Vitamins',
    'hemp seeds': 'Healthy Fats',
    'mango juice beetroot juice': 'Carbohydrates',
    'law fat milk': 'Protein',
    'mushroom': 'Vitamins',
    'or legumes': 'Protein',
    'mushroon': 'Vitamins',
    'beech nuts': 'Healthy Fats',
    'leafy greens': 'Fiber',
    'green juicekale': 'Vitamins',
    'green papper': 'Vitamins',
    'black walnut': 'Healthy Fats',
    'eggs': 'Protein',
    'onion': 'Vitamins',
    'cucumbers': 'Fiber',
    'cattoge cheese': 'Protein',  # Assuming typo for "cottage cheese"
    'apple': 'Carbohydrates',
    'skim milk': 'Protein',
    'cucumber': 'Fiber',  # Same as "cucumbers"
    'aloe vera juice': 'Vitamins',
    'water chestnut': 'Carbohydrates',
    'icebetg lettuce': 'Fiber',  # Assuming typo for "iceberg lettuce"
    'legumes': 'Protein',
    'watermelon juice': 'Carbohydrates',
    'capers': 'Vitamins'
    # Add mappings for all relevant foods in your dataset
}

# **Step 9: Function to Recommend Diets with Nutritional Categories**
def recommend_diet(user_input, model, plan_cols, importance, top_k=30):
    """
    Recommend top-k individual foods based on user input and group them by nutritional category.
    
    Args:
        user_input (dict): User features (e.g., {'Sex': 0, 'Age': 30, ...})
        model (TwoTowerModel): Trained two-tower model
        plan_cols (list): List of plan feature columns
        importance (dict): Feature importance weights
        top_k (int): Number of foods to recommend (default: 30)
    
    Returns:
        dict: Recommended foods grouped by nutritional category
    """
    # Apply importance weights to user input
    for col, weight in importance.items():
        if col in user_input:
            user_input[col] *= weight
    
    # Ensure all user_cols are present in user_input
    missing_cols = [col for col in user_cols if col not in user_input]
    if missing_cols:
        raise ValueError(f"Missing user features: {missing_cols}")
    
    # Convert user input to tensor
    user_features = np.array([user_input[col] for col in user_cols], dtype=np.float32)
    user_tensor = torch.tensor(user_features).unsqueeze(0)
    
    # Identify diet columns from plan_cols
    diet_cols = [col for col in plan_cols if col.startswith('diet_')]
    
    # Create pseudo-plans for each diet component
    pseudo_plans = []
    for diet_col in diet_cols:
        pseudo_plan = np.zeros(len(plan_cols), dtype=np.float32)
        diet_index = plan_cols.index(diet_col)
        pseudo_plan[diet_index] = 1
        pseudo_plans.append(pseudo_plan)
    
    # Convert pseudo-plans to tensor
    pseudo_plan_tensor = torch.tensor(np.array(pseudo_plans), dtype=torch.float32)
    
    # Compute compatibility scores for each pseudo-plan
    model.eval()
    with torch.no_grad():
        scores = model(user_tensor.repeat(len(pseudo_plans), 1), pseudo_plan_tensor).numpy()
    
    # Get top-k indices
    top_indices = np.argsort(scores)[-top_k:][::-1]
    
    # Get the corresponding diet components
    top_diet_components = [diet_cols[i] for i in top_indices]
    
    # Decode into readable food names and group by nutritional category
    recommended_foods_by_category = {}
    for component in top_diet_components:
        # Remove 'diet_' prefix and clean up the string
        food_name = component.replace('diet_', '').replace('Diet_', '').replace('and_', '').replace('_', ' ').strip()
        
        # Get the nutritional category
        category = food_category_mapping.get(food_name, 'Other')
        
        if category not in recommended_foods_by_category:
            recommended_foods_by_category[category] = []
        recommended_foods_by_category[category].append(food_name)
    
    return recommended_foods_by_category



Epoch 1, Train Loss: 0.5426, Val Precision: 0.6495, Val Recall: 0.9480, Val F1: 0.7708, Val AUC: 0.8742
Epoch 2, Train Loss: 0.3981, Val Precision: 0.6821, Val Recall: 0.6962, Val F1: 0.6891, Val AUC: 0.8598
Epoch 3, Train Loss: 0.3624, Val Precision: 0.7645, Val Recall: 0.5627, Val F1: 0.6482, Val AUC: 0.9049
Epoch 4, Train Loss: 0.3387, Val Precision: 0.7059, Val Recall: 0.9280, Val F1: 0.8019, Val AUC: 0.8952
Epoch 5, Train Loss: 0.3339, Val Precision: 0.7256, Val Recall: 0.7587, Val F1: 0.7418, Val AUC: 0.9102
Epoch 6, Train Loss: 0.3272, Val Precision: 0.7146, Val Recall: 0.8621, Val F1: 0.7814, Val AUC: 0.9068
Epoch 7, Train Loss: 0.3170, Val Precision: 0.7220, Val Recall: 0.8216, Val F1: 0.7686, Val AUC: 0.9085
Epoch 8, Train Loss: 0.3174, Val Precision: 0.7058, Val Recall: 0.9868, Val F1: 0.8230, Val AUC: 0.9108
Epoch 9, Train Loss: 0.3100, Val Precision: 0.7157, Val Recall: 0.9750, Val F1: 0.8255, Val AUC: 0.9000
Epoch 10, Train Loss: 0.3080, Val Precision: 0.7334, Val Recall:

In [63]:
# **Step 10: Example User Input and Recommendation**
example_user = {
    'Sex': 0,  # Male
    'Age': 30,
    'Height': 1.75,  # meters
    'Weight': 70,  # kg
    'Hypertension': 0,
    'Diabetes': 0,
    'BMI': 22.86,
    'Fitness Goal_Weight Loss': 1,
    # Add other required features (assuming 0 for simplicity if not specified)
    **{col: 0 for col in user_cols if col not in ['Sex', 'Age', 'Height', 'Weight', 
                                                  'Hypertension', 'Diabetes', 'BMI', 
                                                  'Fitness Goal_Weight Loss']}
}

# Get recommendations for 30 foods
recommended_foods_by_category = recommend_diet(example_user, model, plan_cols, importance, top_k=30)

# Display the categorized recommendations
print("\nRecommended Foods by Nutritional Category:")
for category, foods in recommended_foods_by_category.items():
    print(f"{category}:")
    for food in foods:
        print(f"  - {food}")


Recommended Foods by Nutritional Category:
Fiber:
  - celery
  - carrot
  - broccoli
  - cucumbers
  - cucumber
Other:
  - dairy products
  - fruit juice
  - tomatoes
  - mixed greens
  - roma tomatoes
  - nuts
  - lowfat dairy products
  - carrot juice
  - mixed teff
Vitamins:
  - onion
  - garlic
  - green juicekale
  - green papper
  - bell peppers
  - spinach
  - beetroot juice
Protein:
  - fish
  - law fat milk
  - baru nuts
Carbohydrates:
  - mango juice
  - mango
  - mango juice
  - apple
  - apple juice
  - watermelon juice


In [57]:
# **Step 10: Example User Input and Recommendation**
example_user = {
    'Sex': 1,  # Female
    'Age': 25,
    'Height': 1.50,  # meters
    'Weight': 40,  # kg
    'Hypertension': 0,
    'Diabetes': 1,
    'BMI': 19.00,
    'Fitness Goal_Weight Loss': 0,
    # Add other required features (assuming 0 for simplicity if not specified)
    **{col: 0 for col in user_cols if col not in ['Sex', 'Age', 'Height', 'Weight', 
                                                  'Hypertension', 'Diabetes', 'BMI', 
                                                  'Fitness Goal_Weight Loss']}
}

# Get recommendations for 30 foods
recommended_foods_by_category = recommend_diet(example_user, model, plan_cols, importance, top_k=30)

# Display the categorized recommendations
print("\nRecommended Foods by Nutritional Category:")
for category, foods in recommended_foods_by_category.items():
    print(f"{category}:")
    for food in foods:
        print(f"  - {food}")


Recommended Foods by Nutritional Category:
Vitamins:
  - green papper
  - beetroot juice
  - green juicekale
  - cherry tomatoes
  - bell peppers
  - mushroon
  - beetroot juice
  - spinach
  - garlic
Carbohydrates:
  - mango juice
  - mango juice
Fiber:
  - carrot
  - squash seeds
  - broccoli
Protein:
  - baru nuts
  - fish
  - poultry
  - tofu
  - baru nuts
Other:
  - cattoge cheese
  - onion
  - eggs
  - cucumber
  - apple
  - watermelon juice
  - skim milk
  - cucumbers
  - capers
  - water chestnut
Healthy Fats:
  - hemp seeds
